In [ ]:
import os
import numpy as np
from glob import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split
import cv2
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.layers import Conv2D , BatchNormalization , Activation , MaxPool2D , Input , Dropout , ZeroPadding2D , Conv2DTranspose , Concatenate
from tensorflow.keras.applications import DenseNet201

from tensorflow.keras.models import Model

from tensorflow.keras.applications import InceptionResNetV2
def conv_block(inputs , num_filters):
  x = Conv2D(num_filters , 3 , padding= "same" )(inputs)
  x = BatchNormalization()(x)
  x = Activation("relu")(x)
  x = Dropout(0.3)(x)

  x = Conv2D(num_filters , 3 , padding= "same" )(x)
  x = BatchNormalization()(x)
  x = Activation("relu")(x)
  x = Dropout(0.3)(x)

  return x


def decoder_block(inputs , skip_features , num_filters):

  x = Conv2DTranspose(num_filters , (2, 2) , strides = 2 , padding = "same")(inputs)
  x = Concatenate()([x , skip_features ])
  x = conv_block(x , num_filters)

  return x


In [ ]:

# Even the skip features were extracted for the Decoder purposes to feed in the required Skip Features

def build_DenseNet210_Unet(input_shape):

  ##Input
  inputs = Input(input_shape)

  ## Pre-Trained Encoder

  encoder = DenseNet201(include_top = False , weights = "imagenet" , input_tensor = inputs)

  # Skip Connections which are the Features and we will access these Features and Feed it in the Decoder
  #First Feature is the input shaped image
  s1 = encoder.get_layer("input_1").output                         # 256 * 256

  s2 = encoder.get_layer("conv1/relu").output                       # 128 * 128 

  s3 = encoder.get_layer("pool2_relu").output       # 64 * 64 

  s4 = encoder.get_layer("pool3_relu").output       # 32 * 32

  ## BottleNeck or bridge 
  b1 = encoder.get_layer("pool4_relu").output       # 16 * 16

  ## Decoder
  d1 = decoder_block(b1 , s4 , 512)                                # 32 * 32 * 512 where 512 is the number of Features we extracted 

  d2 = decoder_block(d1 , s3 , 256)                                # 64 * 64

  d3 = decoder_block(d2 , s2 , 128)                                # 128 * 128

  d4 = decoder_block(d3 , s1 , 64)                                # 128 * 128

  ## OUTPUT 

  outputs = Conv2D( 1 ,  1  ,  padding = "same" ,  activation = "sigmoid")(d4)

  ## MODEL 

  model = Model(inputs , outputs ,  name = "DenseNet210")

  return model
  #encoder.summary()

    



In [ ]:
if __name__ == "__main__":
  input_shape = (256 , 256 , 3)

  ## This Model is our Encoder and Bridge Part of our Network and we only need to Write the Decoder part only

  model = build_DenseNet210_Unet(input_shape )
  model.summary()



74842112/74836368 [==============================] - 1s 0us/step
Model: "DenseNet210"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 128, 128, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1/conv[0][0]                 
_______________________

In [ ]:
def load_data(paths , split = 0.15):
  images = []
  masks = []
  numImages = 0
  numMasks = 0

  for path in paths:
    
        
    images.extend(sorted(glob(os.path.join(path , "images/*"))))
    masks.extend(sorted(glob(os.path.join(path , "label/*"))))
    if "training" in path:
      for i in ["00", "01" , "02" , "03" , "04" , "05" , "07", "08" , "09" , "10" , "11" , "12" , "13" , "14" , "16" , "17" , "18" , "19" , "21" , "22" , "24" , "26" , "27" ]:
        images.remove("../input/chagas/ChagasTraining/training-20210525T143718Z-001/training/images/i8{}.xml".format(i))

      n = "training"
    if "Test" in path:
      n= "Test"
    if "Val" in path:

      n= "Validation"

    print("{}PathImagesAre:".format(n),len(images) - numImages)
    print("{}PathImagesAre:".format(n),len(masks) - numMasks)

    numImages = len(images)
    numMasks = len(masks)

  total_size = len(images)
  valid_size = int(split * total_size)
  test_size = int(split * total_size)
  print("Number of images in Total , Validation images and Test Images : " , total_size , valid_size ,test_size  )

  train_x , test_x = train_test_split(images , test_size = test_size , random_state = 42 )
  train_y , test_y = train_test_split(masks , test_size = test_size , random_state = 42 )

  train_x , valid_x = train_test_split(train_x , test_size = valid_size , random_state = 42 )
  train_y , valid_y = train_test_split(train_y , test_size = valid_size , random_state = 42 )

  return (train_x , train_y) , (valid_x , valid_y ) , (test_x , test_y) 

def read_image(path):
  path = path.decode()
  x = cv2.imread(path , cv2.IMREAD_COLOR)
  x = cv2.resize(x , (256 , 256))
  x = x/255.0
  #Size is 256 * 256 * 3

  return x

def read_mask(path):
  path = path.decode()
  x = cv2.imread(path , cv2.IMREAD_GRAYSCALE)
  x = cv2.resize(x , (256 , 256))
  x = x/255.0
  
  #Size is 256 * 256
  x = np.expand_dims(x , axis = -1)
  #Size becames 256 * 256 * 1

  return x

def tf_parse(imagepath , maskpath):
  def _parse(imagepath , maskpath):
    x = read_image(imagepath)
    y = read_mask(maskpath)

    return x , y

  x , y = tf.numpy_function(_parse , [imagepath , maskpath] , [tf.float64 , tf.float64] )
  x.set_shape([256 , 256 , 3])
  y.set_shape([256, 256, 1])

  return x , y 


def tf_dataset( imagepath , maskpath , batch = 8):
  dataset = tf.data.Dataset.from_tensor_slices((imagepath , maskpath))
  dataset = dataset.map(tf_parse)
  dataset = dataset.batch(batch)
  dataset = dataset.repeat()
  return dataset

if __name__ == "__main__":
  paths = ["../input/chagas/ChagasTraining/training-20210525T143718Z-001/training" , "../input/chagas/ChagasTest/Test", "../input/chagas/ChagasValidation/Validation"] 
  (train_x , train_y) , (valid_x , valid_y ) , (test_x , test_y)  =   load_data(paths)

  print("Number of images in Train , Validation images and Test Images : " ,len(train_x) , len(valid_x ) , len(test_x) )

  ds = tf_dataset(test_x , test_y)
  for x, y in ds:
    
    print(x.shape , y.shape)
    break
    


trainingPathImagesAre: 600
trainingPathImagesAre: 600
TestPathImagesAre: 200
TestPathImagesAre: 200
ValidationPathImagesAre: 200
ValidationPathImagesAre: 200
Number of images in Total , Validation images and Test Images :  1000 150 150
Number of images in Train , Validation images and Test Images :  700 150 150
(8, 256, 256, 3) (8, 256, 256, 1)


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model





In [ ]:
# Train 
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard
from tensorflow.keras.metrics import Recall , Precision 

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)


In [ ]:
pip install segmentation_models

     |████████████████████████████████| 50 kB 1.7 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%env SM_FRAMEWORK=tf.keras

env: SM_FRAMEWORK=tf.keras


In [ ]:
if __name__ == "__main__":
    from segmentation_models.losses import bce_jaccard_loss
    from segmentation_models.metrics import iou_score
    np.random.seed(42)
    tf.random.set_seed(42)
    #HYPERPARAMETER


    batch = 8
    lr = 1e-4
    epochs = 60
    opt = tf.keras.optimizers.Adam(lr)
    metrics = ["acc" , Recall() , Precision() , iou_score]
    #Data
    train_dataset = tf_dataset(train_x, train_y, batch=batch)
    valid_dataset = tf_dataset(valid_x, valid_y, batch=batch)

    #model = build_model()
    #model.summary()



    model.compile(loss=bce_jaccard_loss, optimizer=opt, metrics=metrics)

    callbacks = [
        ModelCheckpoint("files/model.h5"),
        ReduceLROnPlateau(monitor='val_iou', factor=0.1, patience=4),
        CSVLogger("../data.csv"),
        TensorBoard(),
        EarlyStopping(monitor='val_iou', patience=20, restore_best_weights=False)
    ]
    #Number Of Batches
    train_steps = len(train_x)//batch
    valid_steps = len(valid_x)//batch

    if len(train_x) % batch != 0:
        train_steps += 1
    if len(valid_x) % batch != 0:
        valid_steps += 1

    model.fit(train_dataset,
        validation_data=valid_dataset,
        epochs=epochs,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=callbacks,
        shuffle = False)


Segmentation Models: using `tf.keras` framework.
Epoch 1/60
88/88 [==============================] - 57s 417ms/step - loss: 1.5851 - acc: 0.7169 - recall: 0.3345 - precision: 0.0196 - iou_score: 0.0149 - val_loss: 1.2414 - val_acc: 0.9840 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - val_iou_score: 0.0124
Epoch 2/60
88/88 [==============================] - 26s 295ms/step - loss: 1.1136 - acc: 0.9829 - recall: 0.6863 - precision: 0.5779 - iou_score: 0.0599 - val_loss: 1.4558 - val_acc: 0.9771 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - val_iou_score: 0.0103
Epoch 3/60
88/88 [==============================] - 26s 293ms/step - loss: 1.0168 - acc: 0.9860 - recall: 0.8308 - precision: 0.6874 - iou_score: 0.1031 - val_loss: 1.1499 - val_acc: 0.9833 - val_recall: 3.3430e-05 - val_precision: 3.3781e-04 - val_iou_score: 0.0099
Epoch 4/60
88/88 [==============================] - 26s 294ms/step - loss: 0.9517 - acc: 0.9875 - recall: 0.8840 - precision: 0.7443 - iou_score: 0.13

In [ ]:
    model.fit(train_dataset,
        validation_data=valid_dataset,
        epochs=30,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=callbacks,
        shuffle = False)


Epoch 1/30
88/88 [==============================] - 27s 308ms/step - loss: 0.1340 - acc: 0.9922 - recall: 0.8868 - precision: 0.9884 - iou_score: 0.8747 - val_loss: 0.4283 - val_acc: 0.9894 - val_recall: 0.6634 - val_precision: 0.9137 - val_iou_score: 0.6205
Epoch 2/30
88/88 [==============================] - 26s 295ms/step - loss: 0.1336 - acc: 0.9922 - recall: 0.8865 - precision: 0.9886 - iou_score: 0.8751 - val_loss: 0.4044 - val_acc: 0.9894 - val_recall: 0.7086 - val_precision: 0.8931 - val_iou_score: 0.6414
Epoch 3/30
88/88 [==============================] - 26s 296ms/step - loss: 0.1365 - acc: 0.9922 - recall: 0.8869 - precision: 0.9863 - iou_score: 0.8725 - val_loss: 0.3890 - val_acc: 0.9893 - val_recall: 0.7423 - val_precision: 0.8765 - val_iou_score: 0.6553
Epoch 4/30
88/88 [==============================] - 26s 294ms/step - loss: 0.1380 - acc: 0.9922 - recall: 0.8871 - precision: 0.9854 - iou_score: 0.8712 - val_loss: 0.4505 - val_acc: 0.9893 - val_recall: 0.6395 - val_precis

In [ ]:
import os
  
# Directory
directory = "RESULT_DenseNet210WithDropout"
  
# Parent Directory path
parent_dir = "./"
  
# Path
path = os.path.join(parent_dir, directory)
  
# Create the directory
# 'GeeksForGeeks' in
# '/home / User / Documents'
os.mkdir(path)

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from tqdm import tqdm
#from data import load_data, tf_dataset
#from train import iou

def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (256, 256))
    x = x/255.0
    return x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (256, 256))
    x = np.expand_dims(x, axis=-1)
    return x

def mask_parse(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask, mask]
    mask = np.transpose(mask, (1, 2, 0))
    return mask

if __name__ == "__main__":
    ## Dataset
    
    batch_size = 8
    #(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data("/content/drive/MyDrive/Test")

    test_dataset = tf_dataset(test_x, test_y, batch=batch_size)

    test_steps = (len(test_x)//batch_size)
    if len(test_x) % batch_size != 0:
        test_steps += 1

    #with CustomObjectScope({'iou': iou}):
    #    model = tf.keras.models.load_model("/content/drive/model.h5")

    #model.evaluate(test_dataset, steps=test_steps)

    for i, (x, y) in tqdm(enumerate(zip(test_x, test_y)), total=len(test_x)):
        x = read_image(x)
        y = read_mask(y)
        y_pred = model.predict(np.expand_dims(x, axis=0))[0] > 0.5
        h, w, _ = x.shape
        white_line = np.ones((h, 10, 3)) * 255.0

        all_images = [
            x * 255.0, white_line,
            mask_parse(y), white_line,
            mask_parse(y_pred) * 255.0
        ]
        image = np.concatenate(all_images, axis=1)
        
        cv2.imwrite(f"./RESULT_DenseNet210WithDropout/{i}.png", image)